## Lets work on creating a programmatic way to receive clip full videos into clips of individual swings

### To generate clipped dataset, we want to utilize:
    1) csv with the appropriate information about original videos, swings
    2) ffmpeg commands that will dictate how the video is decoded/encoded to generate the clips
         - Since videos are from an iphone --> there are some peculiarities to consider: 
             - https://www.perplexity.ai/search/8176d69e-475a-4e7f-ba07-0b762adb7c65
    We'll then create an appropriate clipped folder within folder holding all the full videos from a specific date

In [2]:
#| include: false
from fastai.vision.all import *
from IPython.display import Video
import subprocess
import ffmpeg
import cv2

In [3]:
base_path = '../../../data/full_videos'
swing_days = ['jun8', 'aug9', 'sep14']
files = get_files(f'{base_path}/{swing_days[0]}', extensions='.MOV')
len(files), files[0]

(13, Path('../../../data/full_videos/jun8/IMG_0851.MOV'))

In [137]:
df_jun8 = pd.read_csv(f'{base_path}/{swing_days[0]}/cleaned.csv').reset_index(drop=True)
df_aug9 = pd.read_csv(f'{base_path}/{swing_days[1]}/cleaned.csv').reset_index(drop=True)
df_sep14 = pd.read_csv(f'{base_path}/{swing_days[2]}/cleaned.csv').reset_index(drop=True)
df_jun8.head(1)

,input_file,swing_index,score,start,end,output_file
0,IMG_0848.MOV,0,2,00:20,00:23,IMG_0848_swing_0_score_2.mp4


In [174]:
# just checking that the dataframe columns are the same
assert((df_aug9.columns == df_jun8.columns).sum() == len(df_aug9.columns))
assert((df_aug9.columns == df_sep14.columns).sum() == len(df_aug9.columns))

In [272]:
## All the necessary code to take a dataframe and clip the videos however we need

def clip_videos(df, 
                input_path, 
                crf='18',
                video_encoder='copy',
                debug=False):
    output_folder_path = f'{input_path}/crf_{crf}_vcodec_{video_encoder}'
    for idx in range(len(df)):
        make_clip(input_folder_path=input_path, 
                  output_folder_path=output_folder_path,
                  row=df.iloc[idx],
                 crf=crf,
                 vcodec=video_encoder)
        if debug: break
    assert(len(os.listdir(output_folder_path)) == len(df))

def make_output_filename(row):
    return f'{row.input_file.split(".")[0]}_swing_{row.swing_index}_score_{row.score}.mp4'

def make_clip(input_folder_path, 
              output_folder_path,
              row, 
              time='0:03',
              crf='18',
              vcodec='copy'):
    input_file_name = row.input_file
    output_file_name = make_output_filename(row)
    start = row.start
    input_file_path = f'{input_folder_path}/{input_file_name}'
    output_file_path = f'{output_folder_path}/{output_file_name}'
    if os.path.isdir(output_folder_path) is False:
        os.mkdir(output_folder_path)
    ffmpeg.input(input_file_path, 
                 ss=start)\
        .output(output_file_path, 
                t=time, 
                vcodec=vcodec,
                crf=crf, 
                acodec='aac',
                y=None) \
        .global_args('-movflags', '+faststart') \
        .run()

In [281]:
#clip_videos(df_sep14, input_path=f'{base_path}/{swing_days[2]}', video_encoder='libx264', crf=0)

In [ ]:
## clip_videos(df_aug9, input_path=f'{base_path}/{swing_days[1]}', video_encoder='libx264', crf=0)

In [282]:
! python

Python 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
KeyboardInterrupt
>>> 

In [274]:
#clip_videos(df_aug9, input_path=f'{base_path}/{swing_days[1]}')

In [275]:
#clip_videos(df_sep14, input_path=f'{base_path}/{swing_days[2]}')

## Now some code to pull down the frames

In [283]:
def get_frames(swing_path, 
               resize=True, 
               width=256, 
               height=256,
               debug=False):
    capture = cv2.VideoCapture(swing_path)
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_array = np.empty((frame_count, frame_height, frame_width, 3), dtype=np.uint8)
    idx = 0
    while idx < frame_count:
        ret, frame = capture.read()
        if not ret:
            break
        video_array[idx] = frame
        idx += 1

    capture.release()
    if debug:
        print(video_array.shape)
    video_array = [convert_rgb(frame) for frame in video_array]
    if resize:
        video_array = np.array([resize_frame(frame, width, height) for frame in video_array])
    return video_array

def resize_frame(frame, width=256, height=256):
    return cv2.resize(frame, (width, height))

def convert_rgb(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

def plot_three(image1, image2, image3=None):
    # Create figure with 1 row, 3 columns of subplots
    if image3 is None:
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    else:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Plot each image on its respective axis
    axes[0].imshow(image1)
    axes[0].set_title('Frame 1')
    axes[0].axis('off')  # Remove axis ticks and labels

    axes[1].imshow(image2)
    axes[1].set_title('Frame 2')
    axes[1].axis('off')

    if image3 is not None:
        axes[2].imshow(image3)
        axes[2].set_title('Frame 3 or Diff')
        axes[2].axis('off')

    plt.tight_layout()  # Adjust spacing between subplots
    plt.show()

In [290]:
jun8_files = get_files(f'{base_path}/jun8/crf_18_vcodec_copy')
aug19_files = get_files(f'{base_path}/aug9/crf_18_vcodec_copy')
sep14_files = get_files(f'{base_path}/sep14/crf_18_vcodec_copy')
len(jun8_files), len(aug19_files), len(sep14_files)

(85, 78, 46)

In [310]:
jun_files_18 = get_files(f'{base_path}/jun8/crf_18_vcodec_copy')
jun_files_0 = get_files(f'{base_path}/jun8/crf_0_vcodec_libx264')

In [311]:
jun_files_18[0], jun_files_0[0]

(Path('../../../data/full_videos/jun8/crf_18_vcodec_copy/IMG_0848_swing_0_score_2.mp4'),
 Path('../../../data/full_videos/jun8/crf_0_vcodec_libx264/IMG_0848_swing_0_score_2.mp4'))

In [318]:
aug19_files

(#78) [Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1014_swing_10_score_1.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1023_swing_4_score_5.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1015_swing_10_score_4.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1015_swing_6_score_1.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1018_swing_0_score_4.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1023_swing_0_score_1.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1023_swing_13_score_3.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1017_swing_2_score_4.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1014_swing_5_score_3.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1023_swing_12_score_1.mp4'),Path('../../../data/full_videos/aug9/crf_18_vcodec_copy/IMG_1019_swing_7_score_1.mp4'),Path('../../../data/full_videos/

In [317]:
Video(aug19_files[0], width=200, height=300)

In [316]:
Video(jun8_files[0], width=200, height=300)

In [307]:
#Video??